In [ ]:
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from zumi.personality import Personality
from zumi.protocol import Note 

from zumi.zumi import Zumi
import cv2

import IPython.display
import PIL.Image
import time
import math
import pyzbar.pyzbar as pyzbar
import numpy as np


screen = Screen()
zumi = Zumi()


In [ ]:
#resolution
width = 160 
height = 120

personality = Personality(zumi, screen)
camera = Camera(width,height)
camera.start_camera()

zumi.mpu.calibrate_MPU()
zumi.reset_gyro()


heading = 0
celebrated = False

screen.blink()

        


def calc_distance_to_qr(qr_area):
    #distance = 2594*qr_area**(-0.5)*1.3
    length_of_qr = 3 #3 inches
    area_of_object = length_of_qr**2
    area_detected = qr_area
    k = 0.96 #some constant which we can vary to get better results
    distance = ( k*(area_of_object*width*height)/area_detected )**(1/2)
    return distance


def error_drive(error,k = 0.5):
    #make sure max error is 80
    iterations = abs(int(error*k))
    max_iter = 12
    
    if iterations>max_iter:
        iterations = max_iter
    #if zumi is on the left or right of the object
    if error > 0 and iterations > 1:
        print("left")
        for i in range(iterations):
            zumi.control_motors(i,-i)

    elif error < 0 and iterations > 1:
        print("right")
        for i in range(iterations):
            zumi.control_motors(-i,i)  
    zumi.stop()        
    print("centered error",error , "iterations",iterations)    

    
def error_drive_distance(error,k = 5):
    iterations = abs(int(error*k))
    base_speed = 30
    max_iter = 12
    
    #find current angle
    angle = zumi.update_angles()[2]
    
    if iterations>max_iter:
        iterations = max_iter

    if error < 0 and iterations > 1:
        print("reverse")
        for i in range(iterations):
            zumi.go_reverse(base_speed+i,angle)
            
    elif error > 0 and iterations > 1:
        print("forward")
        for i in range(iterations):
            zumi.go_straight(base_speed+i,angle)
        
    zumi.stop()        
    print("distance error",error , "iterations",iterations)        
     

try: 
    while True:
        #update the angle traveled
        zumi.update_angles()
    
        #keep track of the time
        time_start = time.time()
        
        #take a picture
        frame = camera.capture() 
        
        # Run a function that looks for codes in that frame
        decodedObjects = pyzbar.decode(frame) 

        # If the code finds more than one code...
        if len(decodedObjects) > 0:

            # For each QR code...
            for obj in decodedObjects: 

                #move Zumi depending on location of object
                width_of_qr = obj.rect[2]
                height_of_qr = obj.rect[3]
                cx_of_qr = obj.rect[0]+ width_of_qr/2
                x_middle_of_screen = width/2

                area_rectangle = width_of_qr*height_of_qr
                distance = calc_distance_to_qr(area_rectangle)
                error_distance = distance-7
                error = (x_middle_of_screen-cx_of_qr)
                error_drive(error)
                error_drive_distance(error_distance)

                print("distance",distance)
                print("Message: ", obj.data.decode("utf-8"))

         #convert the image for displaying
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert it to gray
        ret,black_white = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
        
         #display the video
        #IPython.display.display(PIL.Image.fromarray(black_white))
        print("FPS", 1/(time.time()-time_start))
        print("angle ", zumi.angle_list[2])
        IPython.display.clear_output(wait=True) 

finally:
    camera.close()
    zumi.stop()
    print("done")